## 9. Базы данных (с уклоном в PostgreSQL)

![Database](https://raw.githubusercontent.com/amaargiru/pycore/main/pics/09_Database.png)  

### БД и СУБД

База данных (БД) — набор взаимосвязанных данных, хранение, доступ и обработка которых осуществляется посредством [систем управления базами данных (СУБД)](https://en.wikipedia.org/wiki/Database#Database_management_system).

СУБД отвечает за:  
• поддержку языка запросов БД;  
• хранение данных;  
• организацию и оптимизацию процессов записи и извлечения данных.

Отталкиваясь от способа доступа к БД, СУБД можно разделить на три класса:  
• файл-серверные. Файлы хранятся централизованно на сервере, СУБД установлены на клиентских машинах. Примеры — Microsoft Access, Visual FoxPro;  
• клиент-серверные. И файлы данных, и СУБД располагаются централизованно на сервере. Примеры — MySQL, PostgreSQL, Oracle Database, MS SQL Server, MariaDB(форк MySQL);  
• встраиваемые. Встраиваемая СУБД предназначена, как правило, для работы с данными одного конкретного приложения и не рассчитана на работу с внешними запросами. Реализуется, как правило, не в виде самостоятельно инсталлируемого ПО, а в виде подключаемой библиотеки. Самый известный практический пример — [SQLite](https://en.wikipedia.org/wiki/SQLite), остальные встраиваемые СУБД гораздо менее распространены.

Обобщенные требования к РСУБД изложены математиком Эдгаром Коддом в [«12 правилах Кодда»](https://en.wikipedia.org/wiki/Codd%27s_12_rules).

Далее мы сосредоточимся на СУБД PostgreSQL. Также, если не указано иное, при рассмотрении примеров SQL-запросов будет использоваться синтаксис PostgreSQL.


### Транзакция

Остановимся чуть более подробно на механизме транзакций, на требованиях, предъявляемых к транзакциям и на методах соблюдения этих требований.

Формально, [транзакция](https://en.wikipedia.org/wiki/Database_transaction) — неделимая последовательность действий (атомарная операция, группа операций, которые выполняются как единое целое), обеспечивает выполнение либо всех действий из последовательности, либо ни одного. Если в ходе выполнения транзакции произошел сбой, состояние системы должно быть возвращено к исходному, уже выполненные действия должны быть отменены.

Если же говорить неформально, поток транзакций — это динамическое изменение состояний базы данных, схожее с сердцебиением или дыханием у человека. Также как для человека, правильное, согласованное сердцебиение является залогом нормального функционирования; разлад потока транзакций может привести с существенным сбоям и падению базы данных.

Канонический пример — списывание денег с одного счета и зачисление на другой, для чего необходимы два процесса проведения изменений, которые гарантированно должны выполниться или не выполниться вместе. Зачисление денег на один счет без списывания со второго, или наоборот, списывание денег с одного счета без изменений на втором — совершенно недопустимая операция, ставящая под сомнение самое существование жизни, вселенной и всего такого.

В частном случае в группе транзакций может быть одна операция.

Транзакция начинается с команды BEGIN и заканчивается командой COMMIT либо отменяется командой ROLLBACK.

### ACID

Транзакции должны обладать следующими свойствами:

1. **Атомарность** (atomicity). Это свойство означает, что либо транзакция будет зафиксирована в базе данных полностью, т. е. будут зафиксированы результаты выполнения всех ее операций, либо не будет зафиксирована ни одна операция транзакции.

2. **Согласованность** (consistency). Это свойство предписывает, чтобы в результате успешного выполнения транзакции база данных была переведена из одного консистентного состояния в другое консистентное состояние.

3. **Изолированность** (isolation). Во время выполнения транзакции другие транзакции должны оказывать по возможности минимальное влияние на нее.

4. **Долговечность** (durability). После успешной фиксации транзакции пользователь должен быть уверен, что данные надежно сохранены в базе данных и впоследствии могут быть извлечены из нее, независимо от последующих возможных сбоев в работе системы.

Для обозначения всех этих четырех свойств используется аббревиатура ACID. Именно через призму соблюдения требований ACID (или отказа от их части) нужно рассматривать вопросы надежности, предсказуемости и масштабируемости БД.


### Проблемы параллельного доступа с использованием транзакций

Транзакции можно исполнять последовательно или параллельно. И если в первом случае все более-менее понятно и предсказуемо, то в случае параллельного исполнения транзакций возможны следующие проблемы:

**Фантомное чтение** (phantom reads) — одна транзакция в ходе своего выполнения несколько раз выбирает множество строк по одним и тем же критериям. Другая транзакция в интервалах между этими выборками добавляет строки или изменяет столбцы некоторых строк, используемых в критериях выборки первой транзакции, и успешно заканчивается. В результате получится, что одни и те же выборки в первой транзакции дают разные множества строк.

**Неповторяющееся чтение** (non-repeatable read) — при повторном чтении в рамках одной транзакции ранее прочитанные данные оказываются изменёнными.

**«Грязное» чтение** (dirty read) — чтение данных, добавленных или изменённых транзакцией, которая впоследствии не подтвердится (откатится);

**Потерянное обновление** (lost update) — при одновременном изменении одного блока данных разными транзакциями теряются все изменения, кроме последнего.

### Уровни изоляции транзакций

Для борьбы с проблемами, порождаемыми параллельным исполнением транзакций у нас есть соответствующий инструмент — [уровни изоляции транзакций](https://en.wikipedia.org/wiki/Isolation_(database_systems)) — фактически, выбор между скоростью работы и обеспечением согласованности данных, т. к. при выполнении параллельных транзакций в СУБД всегда допускается получение несогласованных данных, и разработчик должен найти баланс между количеством параллельных транзакций и согласованностью данных.

Стандарт SQL-92 определяет шкалу из четырёх уровней изоляции: чтение незафиксированных данных, чтение фиксированных данных, повторяющееся чтение, упорядочивание. Первый из них является самым слабым, последний — самым сильным, каждый последующий включает в себя все предыдущие.

**Чтение незафиксированных данных (read uncommitted)**

Низший (первый) уровень изоляции. Если несколько параллельных транзакций пытаются изменять одну и ту же строку таблицы, то в окончательном варианте строка будет иметь значение, определенное всем набором успешно выполненных транзакций. При этом возможно считывание не только логически несогласованных данных, но и данных, изменения которых ещё не зафиксированы, т. к. транзакции, выполняющие только чтение, при данном уровне изоляции никогда не блокируются. Данные блокируются на время выполнения команды записи, что гарантирует, что команды изменения одних и тех же строк, запущенные параллельно, фактически выполнятся последовательно, и ни одно из изменений не потеряется.

**Чтение фиксированных данных (read committed)**

Большинство СУБД, в частности, Microsoft SQL Server, PostgreSQL и Oracle, по умолчанию используют именно этот уровень. На этом уровне обеспечивается защита от чтения промежуточных данных, тем не менее, в процессе работы одной транзакции другая может быть успешно завершена и сделанные ею изменения зафиксированы. В итоге первая транзакция будет работать с другим набором данных.  
Метод read committed реализуется либо при помощи блокировки данных на чтение во время записи (теряем время), либо на хранении копии данных, снятой до начала записи (теряем ОЗУ).

**Повторяющееся чтение (repeatable read)**

Уровень, при котором читающая транзакция блокирует изменения данных, прочитанных ею ранее. При этом никакая другая транзакция не может изменять данные, читаемые текущей транзакцией, пока та не окончена.

**Упорядочивание (serializable)**

Самый высокий уровень изолированности; транзакции полностью изолируются друг от друга, каждая выполняется так, как будто параллельных транзакций не существует. Только на этом уровне параллельные транзакции не подвержены эффекту «фантомного чтения».

<style>
table th:first-of-type {
    width: 20%;
}
table th:nth-of-type(2) {
    width: 20%;
}
table th:nth-of-type(3) {
    width: 20%;
}
table th:nth-of-type(4) {
    width: 20%;
}
table th:nth-of-type(5) {
    width: 20%;
}
</style>

| Уровень изоляции | Фантомное чтение| Неповторяющееся чтение | «Грязное» чтение | Потерянное обновление |
| :- | :-: | :-: | :-: | :-: |
| Отсутствие изоляции | + | + | + | + |
| Read uncommitted | + | + | + | - |
| Read committed | + | + | - | - |
| Repeatable read | + | - | - | - |
| Serializable | - | - | - | - |

### NoSQL

Перед тем, как углубится в, собственно, Postgres, пройдёмся немного по классу СУБД, объединенных понятием [NoSQL](https://en.wikipedia.org/wiki/NoSQL).

Так же, как рост размера файлов данных предопределил переход от файл-серверных БД к клиент-серверным, так и рост требований к масштабируемости и доступности вызвал появление и развитие NoSQL БД.

Имейте в виду, что сам термин «NoSQL» недостаточно чётко отражает сущность явления, для отказа от использования реляционных БД правильнее было бы использовать термины «No RDBMS» или «NonRel», но исторически прижился именно «NoSQL», на который кроме первоначального акронима «No SQL» повесили дополнительное смысловое значение «Not Only SQL».

Две, пожалуй, основные черты всех NoSQL решений — **отказ от согласованности данных** и относительно лёгкая **масштабируемость**.

В качестве обоснования неизбежности отказа от согласованности данных используется т. н. [теорема CAP](https://en.wikipedia.org/wiki/CAP_theorem), утверждающая, что в любой реализации распределённых вычислений возможно обеспечить не более двух из трёх следующих свойств:  
• согласованность данных (consistency) — во всех вычислительных узлах в один момент времени данные не противоречат друг другу;  
• доступность (availability) — любой запрос к распределённой системе завершается корректным откликом, однако без гарантии, что ответы всех узлов системы совпадают;  
• устойчивость к разделению (partition tolerance) — расщепление распределённой системы на несколько изолированных секций не приводит к некорректности отклика от каждой из секций.

В качестве противопоставления требованиям ACID, рассмотренным выше, для NoSQL сформулированы требования BASE:  
• базовая доступность (basic availability) — подход к проектированию, требующий, чтобы сбой в некоторой части узлов приводил к отказу в обслуживании только для части пользователей при сохранении доступности в большинстве случаев;  
• гибкое состояние (soft state) — возможность жертвовать хранением состояния сессий (промежуточные результаты, информация о контексте), концентрируясь на фиксации обновлений только критичных операций;  
• согласованность в конечном счёте (eventual consistency) — данные могут быть некоторое время рассогласованы, при обеспечении согласования в практически обозримое время.

NoSQL решения были широко распространены примерно до 1970-х, когда на сцене появились реляционные БД. Повторный расцвет популярности NoSQL пришелся на 2010-е, когда Google на примере BigTable показал их эффективность применительно к некоторым типам задач.

Порой выбор между SQL и NoSQL нелегок и требует, конечно, большого опыта работы с БД, у обоих принципов есть свои преимущества и недостатки. В целом, кандидатами на применение NoSQL решений являются системы, объединяющие большое количество данных и некритичность к возможности доступа каждого пользователя к самой последней версии данных. Поисковая система (Google), крупный интернет-магазин (Amazon) — хорошие кандидаты на применение NoSQL. Напротив, банковская или биржевая система, в рамках которой каждому пользователю должны предоставляться самые свежие, актуальные данные требуют применения SQL решений.

### Реляционная модель данных

PostgreSQL является **реляционной** СУБД (РСУБД). Теоретической основой для работы с реляционными базами данных служит раздел математики под названием [реляционная алгебра](https://en.wikipedia.org/wiki/Relational_algebra). [Реляционная модель данных](https://en.wikipedia.org/wiki/Relational_model) (РМД) основана на математическом понятии «отношения» (relation), которое в практическом смысле, на уровне программиста БД, можно толковать как «таблица». Соответственно, реляционную модель данных можно упрощенно воспринимать как «табличную модель данных», т. е. построенную на основе двумерных таблиц, состоящих из строк и столбцов.

Вот небольшой словарик для «перевода» понятий реляционной алгебры на более простой практический «табличный» язык, который может вам пригодиться при чтении обучающей литературы:  
сущность — объекты, содержащиеся в БД (например, сотрудники, клиенты, заказы);  
отношение — таблица;  
атрибут — столбец;  
кортеж — строка (или запись);  
результирующий набор — результат SQL-запроса (как правило, это таблица; в частном случае — таблица из одной строки и одного столбца, т. е., например, число или строка).

Работая с реляционной БД, программисту не нужно заботиться о низкоуровневом доступе к данным, достаточно описать, *что* нужно получить, а *как* именно — описывать не нужно, эту работу берет на себя БД.

Отсутствие низкоуровневого доступа — сильное ограничение, порождающее удобство использования БД (о плюсах ограничений мы поговорим чуть позже, в разделе «Архитектура»). Но это ограничение буквально «ломает мозг» программисту, не имевшему до этого дела с [декларативными](https://en.wikipedia.org/wiki/Declarative_programming) языками программирования.

Когда программист, работающий в императивной парадигме, пытается решить SQL-задачу, руки сами тянутся применить привычный набор инструментов. «Так, проходимся по таблице в цикле, отсеянные значения складываем во временную переменную, потом формируем ответ», но — SQL предлагает совсем другой, непривычный набор возможностей, не предполагая прямого управления ходом выполнения программы: «Скажи, что ты хочешь, а о деталях я позабочусь сам».

Если такой подход для вас внове, не отчаивайтесь; Джо Селко в своей книге «Thinking in Sets» сразу рекомендует готовиться к длительной перестройке процесса мышления: «Я ориентирую студентов на срок около года, полностью посвященного программированию на SQL, только тогда вас может посетить прозрение и вы начнете думать на языке SQL» («I have been telling students that you need about one year of full-time SQL programming before you have an epiphany and start thinking in SQL»).

Так что (продолжая цитировать книгу Селко) потихоньку избавляйтесь от своего «procedural programming mindset», который приводит к «overly complex and inefficient code», и «change the way you think about the problems you solve with SQL programs». Или, по простому, как советуют пользователи Stackoverflow в этом [топике](https://stackoverflow.com/questions/1119505/how-to-think-in-sql), «прекратите думать о построчной обработке, подумайте над решением задачи, основанном на работе со множествами».

### Язык SQL

SQL (structured query language, язык структурированных запросов) — декларативный (описательный, непроцедурный) язык, стандарт для работы с данными во всех реляционных СУБД.  
Операторы SQL традиционно делят на:  
операторы определения данных (data definition language, **DDL**), например, CREATE, ALTER, DROP;  
операторы манипулирования данными (data manipulation language, **DML**), таких как SELECT, INSERT, DELETE;  
операторы управления транзакциями (transaction control language, **TCL**), например, COMMIT, ROLLBACK;  
операторы управления привилегиями доступа (data control language, **DCL**), к примеру, GRANT, REVOKE, DENY.

### Многообразие диалектов СУБД. ANSI SQL.

Прежде чем начать, наконец-то, писать запросы к БД, нам нужно определиться с конкретной СУБД, которую мы будем использовать в дальнейшем (спойлер — как и следует из названия главы, «Базы данных (с уклоном в PostgreSQL)», это будет Postgres). К сожалению, диалекты языка SQL для разных СУБД достаточно сильно различаются и писать универсальные запросы, подходящие ко всем популярным СУБД, у нас не получится, волей-неволей придётся сосредоточиться на одной конкретной реализации.

Общий знаменатель у разных диалектов есть, называется он ANSI SQL и имеет несколько актуальных редакций, от [SQL-92](https://en.wikipedia.org/wiki/SQL-92) до [SQL-2023](https://en.wikipedia.org/wiki/SQL:2023). SQL-92, несмотря на дату публикации, до сих пор вполне актуален и изучение ANSI SQL стоит начать именно с него. К сожалению, ANSI SQL выступает скорее в качестве своеобразного маяка, определяющего разработку СУБД в целом, но не накладывающего жестких ограничений на синтаксис языка.

Существуют попытки создать обучающие руководства на базе ANSI SQL, например, [Wikibook SQL](https://en.wikibooks.org/wiki/Structured_Query_Language), но при практическом изучении лучше сразу ориентироваться на конкретный диалект. ANSI SQL покрывает примерно 90 % возможностей каждого конкретного диалекта, что вроде бы выглядит как достаточно солидный базис, однако в практических сценариях желание использовать для обучения и работы ANSI SQL быстро исчезает, т. к. разница между диалектами слишком велика и попытка решения реальных задач на ANSI SQL приводит к постоянным поискам компромисса и неиспользованию возможностей вашего инструмента.

Учитывая сильное расхождение между диалектами и то, что часть возможностей ANSI SQL не реализована ни в одном из существующих диалектов (как сетуют разработчики Wikibook SQL, 18 мегабайт спецификации на 1400 страницах — не фунт изюму), для реальной работы придется выбрать какой-то один диалект SQL. Но это не такая большая проблема, как кажется, и после изучения одного из диалектов остальные варианты дадутся вам уже гораздо проще.

### Выбор СУБД. SQLite, MySQL, PostgreSQL, SQL Server, Oracle SQL.

Как я уже говорил, в дальнейшем мы будем использовать PostgreSQL. Прежде чем окончательно перейти на Postgres, уделим немного внимания его конкурентам.

### SQLite

Small. Fast. Reliable. Choose any three.  

https://sqlite.org/index.html  
https://github.com/sqlite/sqlite  
https://habr.com/ru/post/149356/  
https://github.com/sqlitebrowser/sqlitebrowser  

Syntax Diagrams https://www.sqlite.org/syntaxdiagrams.html  

Server-less database engine that stores each database into a separate file.

### MySQL

### SQL Server

### Oracle SQL

Перейдём к нашему рабочему инструменту, PostgreSQL.


### Преимущества PostgreSQL

[Каскадные триггеры](https://www.postgresql.org/docs/current/sql-createtrigger.html). Если триггерная функция выполняет команды SQL, эти команды могут заново запускать триггеры. Насколько я знаю, каскадные триггеры есть и в SQL Server.

[hstore](https://www.postgresql.org/docs/current/hstore.html). Возможность создавать и манипулировать данными с функциональностью словаря (dictionary).

[JSONB](https://www.postgresql.org/docs/current/datatype-json.html). Парсинг JSON осуществляется однократно, во время записи. Более медленная однократная запись, но более быстрые многократные чтения. По умолчанию рекомендуется использовать JSONB, а не JSON.

[Range Types](https://www.postgresql.org/docs/current/rangetypes.html). Никаких больше колонок planned_worktime_start и planned_worktime_end и пляски с операторами сравнения для нахождения других строк, у которых интервал, задаваемый этими колонками пересекается с этой строкой.

Другие удобные нативные типы: [interval](https://www.postgresql.org/docs/current/functions-datetime.html), [macaddr](https://www.postgresql.org/docs/current/datatype-net-types.html#DATATYPE-MACADDR) и [другие](https://www.postgresql.org/docs/current/datatype.html), со встроенными методами работы с ними.

[Массивы](https://www.postgresql.org/docs/current/arrays.html) — нарушение 1-й нормальной формы, но когда всё, что необходимо — это сохранить несколько строчек, то добавление отдельной таблицы с перспективой JOIN'а с ней выглядит совсем непривлекательно.

У PostgreSQL полностью транзакционный DDL, т.е. можно в транзакциях менять схему данных, и эти изменения буду транзакционными. Соответственно, возможны миграции без остановки записи.

[PostGIS](https://postgis.net/). Бесплатное расширение для PostgreSQL с открытым исходным кодом для работы с географическими объектами, дополняющее встроенные возможности БД (point, gist). Работает с точками, ломаными линиями, полигонами, растрами, а также использует их для разных операций, например, поиска.

[PL/pgSQL](https://www.postgresql.org/docs/9.6/plpgsql.html). Процедурный язык для PostgreSQL. Функции PL/pgSQL могут использоваться везде, где допустимы встроенные функции. Например, можно создать функции со сложными вычислениями и условной логикой, а затем использовать их при определении операторов или в индексных выражениях.

### Типы данных PostgreSQL

**Целые числа**

smallint 2 байта  
integer 4 байта Самый распространенный целочисленный тип  
bigint 8 байт

**Автоинкрементируемые целые числа**

smallserial 2 байта  
serial 4 байта  
bigserial 8 байт

Фактически, serial — не истинный тип, а "синтаксический сахар" для замены группы SQL-команд.

Синтаксис для создания столбца типа serial таков:

```sql
CREATE TABLE имя-таблицы (имя-столбца serial);
```

Эта команда эквивалентна следующей группе команд:

```sql
CREATE SEQUENCE имя-таблицы_имя-столбца_seq;
CREATE TABLE имя-таблицы
(
имя-столбца integer NOT NULL
DEFAULT nextval('имя-таблицы_имя-столбца_seq')
);
ALTER SEQUENCE имя-таблицы_имя-столбца_seq
OWNED BY имя-таблицы.имя-столбца;
```

Здесь используется функция nextval, которая получает очередное число из последовательности (по сути, из генератора уникальных целых чисел).

**Вещественные числа**

float4 (real) 4 байта 6 знаков после запятой Неточное значение, накопление ошибок при проведении арифметических операций  
float8 (float, double precision) 8 байт 15 знаков после запятой Неточное значение  

Типы данных real и double precision поддерживают специальное значение NaN, которое означает «не число» (not a number). В математике существует такое понятие, как неопределенность. В качестве одного из ее вариантов служит результат операции умножения нуля на бесконечность. Посмотрите, что выдаст в результате PostgreSQL:
SELECT 0.0 * 'Inf'::real;

NaN
(1 строка)

Значение NaN считается равным другому значению NaN, а также что значение NaN считается большим любого другого «нормального» значения, т. е. не-NaN.

Например, можно сравнить значения NaN и Infinity.

```sql
SELECT 'NaN'::real > 'Inf'::real;
```

decimal (numeric) Точное значение. Денежные рассчеты

**Символы**

char строка фиксированной длины, дополняется пробелами  
varchar изменяемой длины с лимитом  
text нелимитируемой длины

Документация рекомендует использовать типы text и varchar, поскольку такое отличительное свойство типа character, как дополнение значений пробелами, на практике почти не востребовано. В PostgreSQL обычно используется тип text.

Для экранирования символов, например, одинарной кавычки или обратной косой черты, их удваивают. Если экранируемых символов слишком много и это затрудняет чтение текста, строку можно одинарных ковычек заключить в двойные символы «$».

PostgreSQL предлагает еще одно расширение стандарта SQL — строковые константы в стиле языка C. Чтобы иметь возможность их использовать, нужно перед начальной одинарной кавычкой написать символ «E». Например, для включения в константу символа новой строки нужно сделать так:

```sql
SELECT E'Hello\nworld!';
```

**Логическое значение**

Boolean (bool) 1 байт

В качестве истинного состояния могут служить следующие значения: TRUE, 't', 'true', 'y', 'yes', 'on', '1'.  
В качестве ложного состояния могут служить следующие значения: FALSE, 'f', 'false', 'n', 'no', 'off', '0'.

При работе с логическими значениями в условии WHERE можно не писать выражение WHERE is_value = 'yes', а достаточно просто указать WHERE is_value.

**Представление даты и времени**

date. По умолчанию используется формат, рекомендуемый стандартом ISO 8601: «yyyy-mm-dd».

Чтобы «сказать» СУБД, что введенное значение является датой, а не простой символьной строкой, нужно испоьзовать операцию приведения типа. В PostgreSQL она оформляется с использованием двойного символа двоеточия и имени того типа, к которому мы приводим данное значение. Важно учесть, что при выполнении приведения типа производится проверка значения на соответствие формату целевого типа и множеству его допустимых значений.

```sql
SELECT 'Sep 01, 2023'::date;
```

Можно использовать команду получения текущей даты:

```sql
SELECT current_date;
```

time  
time with time zone  

Время вводить как в 12-часовом, так и в 24-часовом формате:

```sql
SELECT '22:15:10'::time;

SELECT '10:15:10 pm'::time;
```

Текущее время:

```sql
SELECT current_time;
```

Типы timestamp, time и interval позволяют задать точность ввода и вывода значений. Точность предписывает количество десятичных цифр в поле секунд. Проиллюстрируем эту возможность на примере типа time, выполнив три запроса: в первом запросе вообще не используем параметр точности, во втором назначим его равным 0, в третьем запросе сделаем его равным 3.

SELECT current_time;

timetz
--------------------

19:46:14.584641+03
(1 строка)
SELECT current_time::time( 0 );

time
----------

19:39:45
(1 строка)
SELECT current_time::time( 3 );

time
--------------

19:39:54.085
(1 строка)

timestamp. временная метка — объединение типов даты и времени.  
timestamp with time zone (timestamptz)

Получение текущей временной метки:

```sql
SELECT current_timestamp;
```

Значения временных меток можно округлять с помощью функции date_trunc. Например, для получения текущей временной отметки с точностью до одного часа нужно сделать так:

```sql
SELECT (date_trunc('hour', current_timestamp));
```

Из значений временных отметок можно с помощью функции extract извлекать отдельные поля, т. е. год, месяц, день, число часов, минут или секунд и т. д. Например, чтобы извлечь номер месяца, нужно сделать так:

```sql
SELECT extract('mon' FROM timestamp '2023-11-21 12:35:555.123456');
```

interval Разница между двумя timestamp

Например:

```sql
SELECT '1 year 6 months ago'::interval;
```

или

```sql
SELECT ('2021-08-16'::timestamp - '2021-08-01'::timestamp)::interval;
```

Формат ввода и вывода даты можно изменить с помощью конфигурируемого параметра datestyle. Значение этого параметра состоит из двух компонентов: первый управляет форматом вывода даты, а второй регулирует порядок
следования составных частей даты (год, месяц, день) при вводе и выводе. Текущее значение этого параметра можно узнать с помощью команды SHOW:

SHOW datestyle;

**Массивы**

PostgreSQL позволяет создавать в таблицах такие столбцы, в которых будут содержаться не отдельные значения, а массивы переменной длины. Эти массивы могут быть многомерными и могут содержать значения любого из встроенных типов, а также типов данных, определенных пользователем.

Для указания на то, что это массив, нужно добавить квадратные скобки к наименованию типа данных, при этом задавать число элементов не обязательно.

Пример:

```sql
CREATE TABLE gamers
(
name text,
points integer[]
);

INSERT INTO gamers
VALUES ( 'Ivan', '{ 1, 3, 5, 6, 7 }'::integer[] ),
       ( 'Mike', '{ 1, 2, 5, 7 }'::integer[] ),
       ( 'Jake', '{ 2, 5 }'::integer[] ),
       ( 'Finn', '{ 3, 5, 6 }'::integer[] )
```

Добавление значения в массив:

```sql
UPDATE gamers
SET points = points || 8
WHERE name = 'Jake';
```

**json, jsonb**

Типы JSON предназначены для сохранения в столбцах таблиц базы данных таких значений, которые представлены в формате JSON (JavaScript Object Notation). Существует два типа: json и jsonb. Основное различие между ними заключается в быстродействии. Если столбец имеет тип json, тогда сохранение значений происходит быстрее, потому что они записываются в том виде, в котором были введены. Но при последующем использовании этих значений в качестве операндов или параметров функций будет каждый раз выполняться их разбор, что замедляет работу. При использовании типа jsonb разбор производится однократно, при записи значения в таблицу. Это
несколько замедляет операции вставки строк, в которых содержатся значения данного типа. Но все последующие обращения к сохраненным значениям выполняются быстрее, т. к. выполнять их разбор уже не требуется.

Есть еще ряд отличий, в частности, тип json сохраняет порядок следования ключей в объектах и повторяющиеся значения ключей, а тип jsonb этого не делает. Рекомендуется в приложениях использовать тип jsonb, если только нет каких-то особых аргументов в пользу выбора типа json.

**NULL — отсутствие данных**


Есть также XML, геометрические типы, типы, определяемые пользователем и прочее.

### DDL

### Constraints (ограничения)

```sql
column_constraint is:

[ CONSTRAINT constraint_name ]
{ NOT NULL |
  NULL |
  CHECK ( expression ) [ NO INHERIT ] |
  DEFAULT default_expr |
  GENERATED ALWAYS AS ( generation_expr ) STORED |
  GENERATED { ALWAYS | BY DEFAULT } AS IDENTITY [ ( sequence_options ) ] |
  UNIQUE [ NULLS [ NOT ] DISTINCT ] index_parameters |
  PRIMARY KEY index_parameters |
  REFERENCES reftable [ ( refcolumn ) ] [ MATCH FULL | MATCH PARTIAL | MATCH SIMPLE ]
    [ ON DELETE referential_action ] [ ON UPDATE referential_action ] }
[ DEFERRABLE | NOT DEFERRABLE ] [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]

and table_constraint is:

[ CONSTRAINT constraint_name ]
{ CHECK ( expression ) [ NO INHERIT ] |
  UNIQUE [ NULLS [ NOT ] DISTINCT ] ( column_name [, ... ] ) index_parameters |
  PRIMARY KEY ( column_name [, ... ] ) index_parameters |
  EXCLUDE [ USING index_method ] ( exclude_element WITH operator [, ... ] ) index_parameters [ WHERE ( predicate ) ] |
  FOREIGN KEY ( column_name [, ... ] ) REFERENCES reftable [ ( refcolumn [, ... ] ) ]
    [ MATCH FULL | MATCH PARTIAL | MATCH SIMPLE ] [ ON DELETE referential_action ] [ ON UPDATE referential_action ] }
[ DEFERRABLE | NOT DEFERRABLE ] [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]
```

Constraints указываются при создании или изменении таблицы. Эти правила ограничивают тип данных, которые могут храниться в таблице. При нарушении ограничений действия с данными выполнены не будут.

NOT NULL — значение не может быть NULL;

CHECK — значения столбца должны соответствовать заданным условиям.

DEFAULT — назначает столбцу значение по умолчанию.

UNIQUE — гарантирует уникальность значений в столбце.

PRIMARY KEY

внешний ключ (foreign key)

Exclusion Constraints

### CREATE, создание БД и таблиц

Создание новой БД:

```sql
CREATE DATABASE testdb
    WITH
    OWNER = postgres
    ENCODING = 'UTF8';
```

Создание новых таблиц:
```sql
CREATE TABLE publisher
(
    publisher_id integer PRIMARY KEY,
    org_name varchar(128) NOT NULL,
    address text NOT NULL
);

CREATE TABLE book
(
    book_id integer PRIMARY KEY,
    title text NOT NULL,
    isbn varchar(32) NOT NULL
)
```

PRIMARY KEY (первичный ключ) предназначен для однозначной идентификации каждой записи в таблице и является строго уникальным, две записи таблицы не могут иметь одинаковые значения первичного ключа. Нулевые значения (NULL) в PRIMARY KEY не допускаются. Если в качестве PRIMARY KEY используется несколько полей, их называют составным ключом.

### DROP, удаление таблицы

```sql
DROP TABLE publisher;
DROP TABLE book
```

### ALTER, изменение структуры таблицы

Добавляем колонку:
```sql
ALTER TABLE book
ADD COLUMN fk_publisher_id integer;
```

### FOREIGN KEY, внешний ключ

Сообщаем о том, что эта колонка будет внешним ключом, ссылающимся на publisher_id в другой таблице
```sql
ALTER TABLE book
ADD CONSTRAINT fk_book_publisher
FOREIGN KEY (fk_publisher_id) REFERENCES publisher(publisher_id)
```

Структура таблицы будет приведена к виду, как если бы мы создавали её при помощи следующих команд:
```sql
CREATE TABLE book
(
    book_id integer PRIMARY KEY,
    title text NOT NULL,
    isbn varchar(32) NOT NULL,
    fk_publisher_id integer REFERENCES publisher(publisher_id) NOT NULL
)
```

Мы создали отношение «один ко многим» (одно издательство может издавать множество книг), самое часто используемое отношение в SQL.

### RENAME

### TRUNCATE

### CREATE INDEX 

создаёт индексы в таблице для быстрого поиска/запросов;

Общая информация
Индексы по нескольким столбцам
Уникальные индексы
Индексы на основе выражений
Частичные индексы

Работа с индексами и ограничениями  

Индексы  
Индекс – специальная структура данных, которая связана с таблицей и создаётся на основе её данных. Индексы создаются для повышения производительности функционирования базы данных.

Какие бывают индексы?

В-дерево

хеш

GiST

SP-GiST

GIN

BRIN

По умолчанию команда CREATE INDEX создаёт индексы типа В-дерево (эффективны в большинстве случаев)

Как можно создать индексы?

• Индекс по столбцу (это чистая классика)

• Индекс по нескольким столбцам

• Уникальный индекс

• Индекс на основе выражения

• Частичный индекс

Для создания индекса используется примерно такой синтаксис:

CREATE [UNIQUE] INDEX <index_name> ON <table_name> ( <column_name>, ... ) [STATEMENT] ;
При этом:

для создания уникального индекса может использоваться слово UNIQUE

для создания выражения его записывают в скобках, например для создания выражения проверки индекса на нижний регистр можно написать так:

. . . ( lower( <column_name> ) ) ;
для создания частичного индекса после скобок запись продолжается, например для проверки на величину можно написать так:

. . . ( . . . ) WHERE <column_name> > 1000 ;

### CREATE VIEW, Представления

При работе с базами данных зачастую приходится многократно выполнять одни и те же запросы, которые могут быть весьма сложными и требовать обращения к нескольким таблицам. Чтобы избежать необходимости многократного формирования таких запросов, можно использовать так называемые представления (views). Если речь идет о выборке данных, то представления практически неотличимы от таблиц с точки зрения обращения к ним в командах SELECT.

Упрощенный синтаксис команды CREATE VIEW, предназначенной для создания представлений:

```sql
CREATE VIEW имя-представления [ ( имя-столбца [, ...] ) ]
AS запрос;
```

Полный синтаксис:

```sql
CREATE [ OR REPLACE ] [ TEMP | TEMPORARY ] [ RECURSIVE ] VIEW name [ ( column_name [, ...] ) ]
    [ WITH ( view_option_name [= view_option_value] [, ... ] ) ]
    AS query
    [ WITH [ CASCADED | LOCAL ] CHECK OPTION ]
```

В этой команде обязательными элементами являются имя представления и запрос к базе данных, который и формирует выборку из нее. Если список имен столбцов не указан, тогда их имена «вычисляются» (формируются) на основании текста запроса.

Давайте создадим простое представление. В главе 3 мы решали задачу: подсчитать количество мест в салонах для всех моделей самолетов с учетом класса обслуживания (бизнес-класс и экономический класс). Запрос был таким:

```sql
SELECT aircraft_code,
fare_conditions,
count( * )
FROM seats
GROUP BY aircraft_code, fare_conditions
ORDER BY aircraft_code, fare_conditions;
```

На его основе создадим представление и дадим ему имя, отражающее суть этого представления.

```sql
CREATE VIEW seats_by_fare_cond AS
SELECT aircraft_code,
fare_conditions,
count( * )
FROM seats
GROUP BY aircraft_code, fare_conditions
ORDER BY aircraft_code, fare_conditions;
```

Теперь мы можем вместо написания сложного первоначального запроса обращаться непосредственно к представлению, как будто это обычная таблица.

```sql
SELECT * FROM seats_by_fare_cond;
```

В отличие от таблиц, представления не содержат данных. При каждом обращении к представлению в команде SELECT данные выбираются из таблиц, на основе которых это представление создано.

### Материализованное представление

Упрощенный синтаксис команды CREATE MATERIALIZED VIEW, предназначенной для создания материализованных представлений, таков:

```sql
CREATE MATERIALIZED VIEW [ IF NOT EXISTS ] имя-мат-представления
[ ( имя-столбца [, ...] ) ]
AS запрос
[ WITH [ NO ] DATA ];
```

В момент выполнения команды создания материализованного представления оно заполняется данными, но только если в команде не было фразы WITH NO DATA. Если же она была включена в команду, тогда в момент своего создания представление остается пустым, а для заполнения его данными нужно использовать команду REFRESH MATERIALIZED VIEW.

Материализованное представление очень похоже на обычную таблицу. Однако оно отличается от таблицы тем, что не только сохраняет данные, но также запоминает запрос, с помощью которого эти данные были собраны.

### DML

### INSERT, добавление новых строк

```sql
Insert INTO book
VALUES
(1, 'The Diary of a Young Girl', '0199535566', 1),
(2, 'Pride and Prejudice', '9780307594006', 1),
(3, 'To Kill a Mockingbird', '0446310786', 2),
(4, 'The book of a Gutsy Women: Favorite Stories of Courage and Resilience', '1501178415', 2),
(5, 'War and Peace', '1788886526', 2);

Insert INTO publisher
VALUES
(1, 'Everyman''s Library', 'NY'),
(2, 'Oxford University Press', 'NY'),
(3, 'Grand Central Publishing', 'Washington'),
(4, 'Simon & Schuster', 'Chicago');
```

Проверка правильности добавленных строк (более подробно оператор SELECT будет рассмотрен ниже):
```sql
SELECT *
FROM book
```

и

```sql
SELECT *
FROM publisher
```

WHERE и HAVING могут использоваться в одном запросе, при этом необходимо учитывать порядок исполнения SQL запроса:  
1. FROM  
2. ON  
3. JOIN  
4. WHERE  
5. GROUP BY  
6. WITH CUBE / WITH ROLLUP  
7. HAVING  
8. Оконные функции  
9. SELECT  
10. DISTINCT  
11. UNION / UNION ALL / INTERSECT / EXCEPT  
12. ORDER BY  
13. TOP / LIMIT / OFFSET  

Сначала определяется таблица, из которой выбираются данные (FROM);  
затем из этой таблицы отбираются записи в соответствии с условием WHERE;  
выбранные данные агрегируются (GROUP BY);  
из агрегированных записей выбираются те, которые удовлетворяют условию после HAVING.

Только потом формируются данные результирующей выборки, как это указано после SELECT (вычисляются выражения, присваиваются имена и пр.). Результирующая выборка сортируется в соответствии с условием, указанным после ORDER BY.  

Знание порядка исполнения SQL запроса важно для того, чтобы понять, почему, например, внутри WHERE невозможно использовать имена выражений из SELECT; просто SELECT выполняется компилятором позже, чем WHERE, поэтому ему неизвестно, какое выражение там описано.

### SELECT, FROM, WHERE

Команда SELECT получает записи из базы данных по определенному условию, которое задается с помощью команды WHERE.

Синтаксис:  
SELECT * FROM имя_таблицы;  
SELECT * FROM имя_таблицы WHERE условие;  
SELECT поле1, поле2... FROM имя_таблицы WHERE условие;  

Полный синтаксис команды SELECT:  
SELECT  
    [STRAIGHT_JOIN] [SQL_SMALL_RESULT] [SQL_BIG_RESULT] [SQL_BUFFER_RESULT]  
    [SQL_CACHE | SQL_NO_CACHE] [SQL_CALC_FOUND_ROWS] [HIGH_PRIORITY]  
    [DISTINCT | DISTINCTROW | ALL]  
select_expression,...  
[INTO {OUTFILE | DUMPFILE} 'file_name' export_options]  
[FROM table_references  
[WHERE where_definition]  
[GROUP BY {unsigned_integer | col_name | formula} [ASC | DESC], ...]  
[HAVING where_definition]  
[ORDER BY {unsigned_integer | col_name | formula} [ASC | DESC], ...]  
[LIMIT [offset,] rows | rows OFFSET offset]  
[PROCEDURE procedure_name(argument_list)]  
[FOR UPDATE | LOCK IN SHARE MODE]]  

### LIKE

### ORDER BY

Сортировка данных в порядке возрастания (ASC) или убывания (DESC).

```
SELECT * FROM user ORDER BY name DESC;
```

### COUNT, SUM, AVG, MAX, MIN, агрегирующие функции

### GROUP BY, HAVING, группировка и фильтрация данных

### JOIN, объединение данных из нескольких таблиц

#### INNER JOIN

Если совсем кратко, то INNER JOIN не пересечение множеств, как нарисовано, а, скорее, перемножение с условием.

#### FULL OUTER JOIN

#### LEFT JOIN

#### RIGHT JOIN

#### CROSS JOIN, перекрестное соединение

Создает набор строк, где каждая строка из одной таблицы соединяется с каждой строкой из второй таблицы.

Self join

Такой вопрос тоже может прозвучать на собеседовании по SQL. Это выражение используется для того, чтобы таблица объединилась сама с собой, словно это две разные таблицы. Чтобы такое реализовать, одна из таких «таблиц» временно переименовывается.

Например, следующий SQL-запрос объединяет клиентов из одного города:

```
SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City
FROM Customers A, Customers B
WHERE A.CustomerID <> B.CustomerID
AND A.City = B.City
ORDER BY A.City;
```

### UPDATE, изменение существующих строк

### DELETE, удаление строк

Операция, которая удаляет записи из таблицы, соответствующие заданному условию. При этом создаются логи удаления, то есть операцию можно отменить.

```
DELETE FROM table_name WHERE condition;
```

### FROM

### SET

### Вложенные запросы

SQL позволяет создавать вложенные запросы. Вложенный запрос – это запрос, размещенный внутри другого запроса SQL.

Вложенный запрос используется для выборки данных, которые будут использоваться в условии отбора записей основного запроса. Его применяют для:  
сравнения выражения с результатом вложенного запроса;  
определения того, включено ли выражение в результаты вложенного запроса;  
проверки того, выбирает ли запрос определенные строки.  

Вложенный запрос имеет следующие компоненты:  
ключевое слово SELECT, после которого указываются имена столбцов или выражения (чаще всего список содержит один элемент),  
ключевое слово FROM и имя таблицы, из которой выбираются данные,  
необязательное предложение WHERE,  
необязательное предложение GROUP BY,  
необязательное предложение HAVING.

опросы на собеседованиях очень часто бессистемные, они могут быть просто о том, что вопрошающему конкретно сейчас пришло в голову, и он вообще не факт, что сам умеет спрашивать о главном, а не второстепенном. Приведу только один пример у вас — про индексы. Вас, видимо, спрашивали о том, какие они бывают и каким синтаксисом создаются. Но поверьте, это совсем не самое главное из того, что стоит знать про индексы. Гораздо важнее понимать их суть, что они из себя представляют внутри — чтобы понимать, почему какой-то запрос индексом ускоряется, а какой-то нет.  

Изучение индексов для ускорения поиска данных в таблице  
Изучение ограничений для защиты данных и обеспечения целостности таблицы

Работа с представлениями и хранимыми процедурами  
Изучение представлений для создания виртуальных таблиц на основе запросов  
Изучение хранимых процедур для создания пользовательских функций и процессов

Продвинутые SQL скиллы:  
MVCC
Оптимизация запросов — знание структуры таблиц и индексов, а также понимания того, как оптимизировать запросы для улучшения производительности.  

Повышение производительности.
Основные понятия.
Методы просмотра таблиц.
Методы формирования соединений наборов строк.
Управление планировщиком.
Оптимизация запросов.

Работа с большими объемами данных — управление партиционированием, кластеризацией и другими методами для обработки и анализа больших объемов данных.  
Использование аналитических функций — использование функций, таких как RANK, ROW_NUMBER, LAG и LEAD, для выполнения сложных аналитических запросов.  
Работа с временными рядами данных — использование функций временных рядов, таких как DATE_TRUNC, DATE_PART и WINDOW функций, для анализа и управления временными рядами данных.  
Работа с географическими данными — использование специальных функций, таких как ST_Distance, ST_Within и ST_Intersection, для анализа и управления географическими данными.  
Работа с хранилищами данных — использование функций ETL (Extract, Transform, Load) для извлечения, преобразования и загрузки данных в хранилища данных.  
Работа с процедурами и триггерами — создание и управление процедурами и триггерами для автоматизации задач и обеспечения целостности данных.  
Работа с реляционной алгеброй — использование различных операторов, таких как JOIN, UNION, INTERSECT и EXCEPT, для выполнения сложных запросов.  
Работа с индексами — создание и управление индексами для улучшения производительности запросов.  
Работа с безопасностью данных — управление доступом к данным и защиту данных от несанкционированного доступа.  

Оконные функции


### Вложенные транзакции

Механизм, который неявно задействован при создании точек сохранения и при обработке исключений.

Что такое курсор и зачем он нужен?  
Что делает оператор JOIN, какие виды бывают?  
Что делает оператор HAVING, примеры?  
В каких случаях вы бы предпочли нереляционную БД?  
Что такое функциональный индекс?  

### VACUUM

Команда VACUUM высвобождает пространство, занимаемое «мертвыми» кортежами, что актуально для часто используемых таблиц. При обычных операциях в Postgres кортежи, удаленные или устаревшие в результаты обновления, физически не удаляются, а сохраняются в таблице до очистки.

### EXPLAIN, EXPLAIN ANALYZE

EXPLAIN ANALYZE – в отличие от просто EXPLAIN не только показывает план выполнения запроса, но и непосредственно выполняет запрос и показывает реальное время выполнения.

### Server side cursor

Способ работы с результатом запроса в базу данных, который позволяет не загружать весь объем данных в память, позволяет работать с большими объемами данных. Дополнительно углубленно можно поговорить про особенности работы в связке с pgbouncer.

Это любители mysql принесли 8 главу своей документации, где эта штука у них называется optimizer и состоит из нескольких частей (query planner, собственно query optimizer и т.д.)

В postgresql (и других базах) это называют planner и в русскоязычном IT в аспекте БД (не только postgres) принято слово именно что "планировщик". Ну а то, что некоторые любители mysql словом "планировщик" с давних пор называют почему-то event/job scheduler, вас в общем случае волновать не должно, это у них там своя атмосфера.

Это всё издержки переноса англоязычной терминологии на русский язык. Каким образом так случилось, что в русском scheduler стал планировщиком, я внятно объяснить не могу.

Schedule — это вообще-то график (дежурств), расписание (действий). Планирование как составление плана действий, здесь отсутствует, расписание — продукт планирования, а не процесс. И scheduler в этом смысле — это исполнитель расписания, а не его составитель.

Plan — это в некотором смысле алгоритм, последовательность шагов при исполнении какого-либо действия (из, например, расписания).

Optimize — в общем случае улучшение этого самого Plan.

Подытожим:

scheduler — хранит таблицу, расписание выполнения задач, и обеспечивает их запуск по этому расписанию;

planner — составляет план действий для исполнения конкретного запроса (а EXPLAIN его, этот план, показывает);

optimizer — часть planner'а (может и отсутствовать). Например, переупорядочивает действия, исключает повторные действия, и т.п.

Таким образом получается, что именно в PostgreSQL всё как раз таки и называется правильно.

### Источники

[Документация по PostgreSQL](https://www.postgresql.org/docs/).

Е. П. Моргунов. [PostgreSQL. Основы языка SQL](https://edu.postgrespro.ru/sql_primer.pdf).  
